In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
import requests

# firefox 

In [4]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
#     driver = webdriver.Firefox()

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20130406 Firefox/23.0


<ipython-input-4-5315509cced7>:7: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


In [5]:
driver.get('https://inrest.com/product-category/31/25/8/')

In [9]:
driver.get('https://inrest.com/product-category/31/25/24/')

# get url products

In [6]:
list_product = []

In [10]:
product_urls = driver.find_elements_by_xpath('//a[@class="product-image-link"]')
len(product_urls)

46

In [11]:

for product in product_urls:
    list_product.append(product.get_attribute('href'))
len(list_product)

101

# Traitement

In [51]:
df = None

In [63]:
df1 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/inrest/Inrest_update.xlsx')
df2 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/inrest/Inrest_update1.xlsx')
df3 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/inrest/Inrest_update2.xlsx')







In [59]:
df2.shape

(176, 41)

In [58]:
df1.shape

(466, 41)

In [65]:
df = pd.concat([df, df3], ignore_index=True)

In [57]:
df.shape

(642, 41)

In [66]:
df.to_excel('clean_data1.xlsx')

In [69]:
color_list = [
    'أبيض',
    'بيج',
    'رصاصي',
    'زهري',
    'عودي',
    'كريمي',
    'أخضر',
    'زهري غامق',
    'زهري فاتح',
    'أزرق',
    
]

In [70]:
df['free_colors'] = ''
for color in color_list:
    df.loc[df['name'].str.contains(color), 'free_colors'] = color

0       أبيض
1        بيج
2      رصاصي
3       زهري
4       عودي
       ...  
97      أخضر
98          
99          
100         
101         
Name: free_colors, Length: 102, dtype: object

In [624]:
df['out_of_stock_qty'] = df['out_of_stock_qty'].str.replace('متوفر في المخزون', '').str.strip()

In [72]:
df['qty'] = df['is_in_stock']


In [73]:
df['is_in_stock'] = 1

In [74]:
df['allow_backorders'] = 1

In [75]:
df['product_online'] = 1

In [78]:
df['out_of_stock_qty'] = -5

In [632]:
df.loc[df['out_of_stock_qty'] == 0, 'estimated_delivery_enable'] = 'Static Text'

In [79]:
df['estimated_delivery_enable'] = 'Static Text'

In [80]:
df.to_excel('toto.xlsx')

In [635]:
df.drop_duplicates(subset=['sku'], inplace=True)

In [636]:
df.to_excel('Update_clean.xlsx')

In [637]:
df['allow_backorders'] = df['is_in_stock']
df['estimated_delivery_enable'] = 'Static Text'

In [638]:
df['url_key'] = df['url_key'].str.replace(', ', '-')
df['url_key'] = df['url_key'].str.replace('/', '-')
df['url_key'] = df['url_key'].str.replace('"', '-')
df['url_key'] = df['url_key'].str.replace('%', '-')
df['url_key'] = df['url_key'].str.replace('+', '-', regex=False)

In [642]:
df.loc[(df['raw_materials'] == 0) | (df['raw_materials'].isnull()), 'raw_materials'] = '__EMPTY__VALUE'

In [643]:
df.loc[(df['weight'] == 0) | (df['weight'].isnull()), 'weight'] = '__EMPTY__VALUE'

In [644]:
df.loc[(df['free_colors'] == 0) | (df['free_colors'].isnull()), 'free_colors'] = '__EMPTY__VALUE'

In [645]:
df.to_excel('toto5.xlsx')

# split name

In [646]:
df[['ts_dimensions_width','ts_dimensions_length', 'ts_dimensions_height']] = df['product_size'].str.split("×",expand=True,)

In [651]:
df[['toto','hertz']] = df['voltage'].str.split("فولت",expand=True,)

In [653]:
df['voltage'] = df['toto']

In [654]:
df.to_excel('toto.xlsx')

In [655]:

df.loc[(df['ts_dimensions_width'] == 0) | (df['ts_dimensions_width'].isnull()), 'ts_dimensions_width'] = '__EMPTY__VALUE__'
df.loc[(df['ts_dimensions_height'] == 0) | (df['ts_dimensions_height'].isnull()), 'ts_dimensions_height'] = '__EMPTY__VALUE__'
df.loc[(df['ts_dimensions_length'] == 0) | (df['ts_dimensions_length'].isnull()), 'ts_dimensions_length'] = '__EMPTY__VALUE__'



In [656]:
df['allow_backorders'] = df['is_in_stock']
df['estimated_delivery_enable'] = 'Static, Text'

In [657]:
df['small_image'] =  df['base_image']
df['swatch_image'] =  df['base_image']
df['thumbnail_image'] =  df['base_image']
df['attribute_set_code'] = 'Default'
df['product_websites'] = 'base'

In [658]:
df['price'] 

0    1,364
0    3,449
0    4,499
0    4,399
0    2,699
     ...  
0      574
0    1,021
0      963
0    2,299
0    2,442
Name: price, Length: 71, dtype: object

In [659]:
def toto_clean(name):
    df[name] = df[name].str.replace('•', '-')
    df[name] = df[name].str.replace(':', '-')
    df[name] = df[name].str.replace('/', '-')
    df[name] = df[name].str.replace('"', '-')
    df[name] = df[name].str.replace('%', '-')
    df[name] = df[name].str.replace('&', '-')
    df[name] = df[name].str.replace('?', '-', regex=False)
    df[name] = df[name].str.replace('(', '-', regex=False)
    df[name] = df[name].str.replace(')', '-', regex=False)
    df[name] = df[name].str.replace('{', '-', regex=False)
    df[name] = df[name].str.replace('}', '-', regex=False)
    df[name] = df[name].str.replace("'", '-', regex=False)
    df[name] = df[name].str.replace('*', 'X', regex=False)
    df[name] = df[name].str.replace('#', '-')
    df[name] = df[name].str.replace('”', '-')
    df[name] = df[name].str.replace('!', '-')
    df[name] = df[name].str.replace('”', '-')
    df[name] = df[name].str.replace('=', '-')
    df[name] = df[name].str.replace('+', '-', regex=False)
    df[name] = df[name].str.replace('،', '-', regex=False)
    df[name] = df[name].str.replace('عرض أقل', '', regex=False)
    df[name] = df[name].str.replace('،', '-', regex=False)
    
def ampty_value(name):
    df.loc[df[name].isnull(), name] = '__EMPTY__VALUE__'
    
    
def price_num(name):
    try:
        df[name] = df[name].str.replace('-', '')
        df[name] = df[name].str.replace(',', '')
    except:
        pass
    df.loc[df[name] == '__EMPTY__VALUE__', name] = 0
    
    df[name] = pd.to_numeric(df[name])
    df.loc[df[name] == 0, name] = '__EMPTY__VALUE__'
def clean_punct(name):
    for c in string.punctuation:
        df[name] = df[name].str.replace(c, '-', regex=False)

In [660]:
def parse_name(text):
    new_text = []
    for i, t in enumerate(text):
        if i == 0:
            new_text.append(t.title())
        else:
            new_text.append(t.lower())
    return ''.join(new_text)

In [661]:
toto_clean('name')

In [662]:
toto_clean('description')

In [663]:
df['short_description'] = df['description']

In [664]:
df.loc[df['short_description'].isnull(), 'short_description'] = df['description']

In [112]:
df['price'] = df['price'].str.replace(',', '')
df['special_price'] = df['special_price'].str.replace(',', '')

<ipython-input-112-934e9ae663de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].str.replace(',', '')
<ipython-input-112-934e9ae663de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['special_price'] = df['special_price'].str.replace(',', '')


In [666]:
df['price'] = pd.to_numeric(df['price'])
df['special_price'] = pd.to_numeric(df['special_price'])

In [667]:
# check if df price is correct si ils ne sont pas changer

In [113]:
df['cost'] = df['price']

<ipython-input-113-536ba60cc8d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost'] = df['price']


In [669]:
df['price'] = df['price'] * 1.05

In [83]:
df['special_price'] = ''

In [82]:
df['supplier'] = 'INS'

In [121]:
df.to_excel('toto1.xlsx')

In [84]:
df['sku'] = 'INS-' + df['sku']

In [86]:
df['url_key'] = df['sku'] + '-' + df['name']

# ADD cats

In [673]:
cats = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/tamkeenstores/categories/Cats_tam_keenstores.xlsx')

In [674]:
list_cats = []
for index, row in cats.iterrows():
    list_cats.append([ row['cat'], row['id']])

In [675]:
for ll in list_cats:
    df.loc[df['categories3']== ll[0], 'categories'] = ll[1]

In [676]:
df.loc[df['categories'].isnull()]

,sku,store_view_code,attribute_set_code,product_websites,product_type,name,description,short_description,url_key,link_url,...,news_from_date,news_to_date,estimated_delivery_enable,estimated_delivery_text,supplier,ts_dimensions_width,ts_dimensions_length,ts_dimensions_height,hertz,toto


In [677]:
df.loc[df['description'].isnull()]

,sku,store_view_code,attribute_set_code,product_websites,product_type,name,description,short_description,url_key,link_url,...,news_from_date,news_to_date,estimated_delivery_enable,estimated_delivery_text,supplier,ts_dimensions_width,ts_dimensions_length,ts_dimensions_height,hertz,toto


In [678]:
df.drop(df[df['description'].isnull()].index, inplace=True)

In [679]:
df.drop(df[df['categories'].isnull()].index, inplace=True)

In [680]:
df.to_excel('toto4.xlsx')

# ADD brand

In [681]:
cols = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/tamkeenstores/categories/brand_tam_keenstores.xlsx')

In [682]:
list_cols = []
for index, row in cols.iterrows():
    list_cols.append([ row['brand'], row['id']])

In [87]:
df['visibility'] = 'Catalog, Search'

In [88]:
df['tax_class_name'] = 'Taxable Goods'

In [89]:
df['product_type'] = 'simple'

In [90]:
two_month = datetime.now() + timedelta(days=60)
two_month = two_month.strftime("%m/%d/%Y")
today = datetime.today().strftime("%m/%d/%Y")

In [91]:
today

'10/15/2021'

In [92]:
df['news_from_date'] = today

In [93]:
df['news_to_date'] = two_month

In [690]:
for col in list_cols:
    df.loc[df['mgs_brand'] == col[0], 'mgs_brand'] = col[1]

In [691]:
df['out_of_stock_qty'] = df['out_of_stock_qty'].astype(str)

In [692]:
df['out_of_stock_qty'] = '-' + df['out_of_stock_qty']

In [693]:
df['qty'] = 0

In [694]:
df.loc[df['mgs_brand'].isnull(), 'mgs_brand'] = '__EMPTY__VALUE__'

In [96]:
df['manufacturer'] = '‫مستوردة‬'

In [97]:
df['estimated_delivery_text'] = 'التوصيل خلال 5-6 ايام عمل'

In [98]:
df['estimated_delivery_enable'] = 'Static Text'

In [100]:
df.to_excel('Insert_update_16_09.xlsx')

In [99]:
df['sku number only'] = df['sku'].str.replace('INS-', '')

In [105]:
df['product_websites'] = 'base'

In [106]:
df['attribute_set_code'] = 'Default'

In [107]:
df['store_view_code'] = ''

In [103]:
df['categories'] = '56,43054,113'

In [110]:
list_ampty = [
    
         'special_price', 
         'barcode',
         'no_of_peices',
         'duvit_info',
         'bed_sheet1_info',
         'pillow_size',
         'small_pillow_info',
        'fabric_type',
         'stuffing_material',
    'free_colors'
]
for toto in list_ampty:
    df.loc[(df[toto] == '') | (df[toto].isnull()), toto] = '__EMPTY__VALUE__'

/home/wafistos/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [108]:
df = df[[ 'sku number only',
        'sku', 
         'store_view_code',
         'attribute_set_code',
         'product_websites',
         'product_type',
         'name',   
         'description', 
         'short_description', 
         'link_url', 
         'categories1', 
         'categories2', 
         'categories3', 
         'categories', 
         'free_colors',
         'url_key',
         'cost',  
         'price',
         'special_price', 
         'barcode',
         'no_of_peices',
         'duvit_info',
         'bed_sheet1_info',
         'pillow_size',
         'small_pillow_info',
        'fabric_type',
         'stuffing_material',
         'visibility',
         
         'tax_class_name', 
         'manufacturer',
         'news_from_date', 
         'news_to_date', 
         'qty', 
         'out_of_stock_qty', 
         'allow_backorders', 
         'is_in_stock', 
         'estimated_delivery_enable',
         'estimated_delivery_text',
         'supplier',   
        ]]

In [3]:
df= pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/tamkeenstores/testtoto.xlsx')

In [114]:
df.to_excel('Insert_update.xlsx')

In [12]:
df.loc[df['manufacturer'].isnull(), 'manufacturer'] = ''

In [14]:
df.loc[df['manufacturer'].str.contains('السعة')]

,Unnamed: 0,Unnamed: 0.1,sku,store_view_code,attribute_set_code,product_websites,product_type,name,description,short_description,...,out_of_stock_qty,allow_backorders,visibility,tax_class_name,manufacturer,news_from_date,news_to_date,estimated_delivery_enable,estimated_delivery_text,supplier
413,413,0,ZMK1705B,NaN,NaN,NaN,NaN,زيترست غلاية، 1.7لتر ، 1800-2150 وات – أزرق,"السعة = (1,7) لتر – القوة (1800 – 2150) وات- ا...","السعة = (1,7) لتر – القوة (1800 – 2150) وات- ا...",...,غير متوفر في المخزون,NaN,NaN,NaN,"السعة = (1,7) لتر – القوة (1800 – 2150) وات- ا...",NaN,NaN,NaN,NaN,NaN
414,414,0,ZMK17SG,NaN,NaN,NaN,NaN,زيترست غلاية ، 1.5لتر ، 1850-2200 وات – ذهبي,"السعة = (1,5) لتر- القوة (1850 – 2200) وات- ال...","السعة = (1,5) لتر- القوة (1850 – 2200) وات- ال...",...,غير متوفر في المخزون,NaN,NaN,NaN,"السعة = (1,5) لتر- القوة (1850 – 2200) وات- ال...",NaN,NaN,NaN,NaN,NaN
415,415,0,ZMK1705G,NaN,NaN,NaN,NaN,زيترست غلاية، 1.7لتر ، 1800-2150 وات – أخضر,"السعة = (1,7) لتر- القوة (1800 – 2150) وات- ال...","السعة = (1,7) لتر- القوة (1800 – 2150) وات- ال...",...,غير متوفر في المخزون,NaN,NaN,NaN,"السعة = (1,7) لتر- القوة (1800 – 2150) وات- ال...",NaN,NaN,NaN,NaN,NaN


In [15]:
df.drop(df[df['manufacturer'].str.contains('السعة')].index, inplace=True)

In [ ]:
df.drop(df[df.score < 50].index, inplace=True)

In [156]:
brands = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/tamkeenstores/categories/brand_tam_keenstores.xlsx')

In [158]:
list_brands = []
for index, row in brands.iterrows():
    list_brands.append('العلامة التجارية: ' + row['brand'])

In [ ]:
# Dealete brand from description 

In [168]:
for tt in list_brands:
    
    df.loc[ df['description'].str.contains(str(tt)),  'description' ] = df['description'].str.replace(tt, '', regex=False)

In [181]:
df.to_excel('pap222.xlsx')

# Fonction bs4

In [532]:
def scrape_all_categories(pat):
    list_urls_categories = []
    
    print('URL:', pat['url'])
    # proxy = '64.124.38.140:8080'
#     for tt in PROXY_LIST:
#         print('proxy', tt)
#         try:
#             r = requests.get(pat['url'], proxies={'http': tt, 'https': tt}, timeout=10 )
#             if r.status_code == 200:
#                 print(r.status_code)
#                 break
#         except:
#             pass
        
        
    r = requests.get(pat['url'])
    soup  = BeautifulSoup(r.text, 'html.parser')
    
    urls = soup.find_all('div', {'class': 'mf-product-thumbnail'})
    len(urls)
    cat1 = pat['cat1']
    cat2 = pat['cat2']
    cat3 = pat['cat3']
    for url in urls:
        list_urls_categories.append({ 'url': url.find('a')['href'],
                                        'cat1': cat1,
                                        'cat2': cat2,
                                        'cat3': cat3,
                                    })
    return list_urls_categories

In [13]:
products = soup.find_all('div', {'class': 'product-element-top'})
len(products)

12

# Scrape products

In [12]:
urls_products = [
    'https://inrest.com/product/%d9%85%d9%81%d8%b1%d8%b4-%d9%82%d8%b7%d9%86-%d8%b3%d8%a7%d8%aa%d8%a7%d9%86-%d9%85%d9%88%d9%81-%d9%81%d8%a7%d8%aa%d8%ad/',
    'https://inrest.com/product/%d8%ba%d8%b7%d8%a7%d8%a1-%d8%b3%d8%b1%d9%8a%d8%b1-%d9%86%d9%81%d8%b1%d9%8a%d9%86-%d8%aa%d8%a7%d9%8a%d9%84%d9%88%d8%b1/',
    'https://inrest.com/product/%d8%ba%d8%b7%d8%a7%d8%a1-%d8%b3%d8%b1%d9%8a%d8%b1-%d9%85%d9%81%d8%b1%d8%af-%d9%85%d9%8a%d8%b1%d8%af%d9%8a%d8%a7%d9%86/',
    'https://inrest.com/product/%d9%85%d9%81%d8%b1%d8%b4-%d8%a7%d9%86-%d8%b1%d9%8a%d8%b3%d8%aa-%d9%81%d9%86%d8%af%d9%82%d9%8a-%d9%85%d9%81%d8%b1%d8%af-%d8%b1%d8%b5%d8%a7%d8%b5%d9%8a/',
    'https://inrest.com/product/%d8%ba%d8%b7%d8%a7%d8%a1-%d8%b3%d8%b1%d9%8a%d8%b1-%d9%86%d9%81%d8%b1%d9%8a%d9%86-%d8%a5%d9%8a%d9%84%d9%88%d8%b3/',
    
]

In [58]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def scrape_product(urls):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    cookies = {'session': '134-8228575-0334220'}
    url = urls
    r = requests.get(url, headers=headers, cookies=cookies,  verify=False, timeout=60)

    soup  = BeautifulSoup(r.text, 'html.parser')
    name = soup.find('h1', {'itemprop': 'name'}).text.strip()
    price = soup.find('p', {'class': 'price'}).find('span', {'class': 'woocommerce-Price-amount amount'}).text.replace('ر.س','').strip()
    in_stock = soup.find('p', {'class': 'stock in-stock'}).text.replace('متوفر في المخزون', '').strip()
    sku = soup.find('span', {'class': 'sku'}).text.strip()
    def extract_detail(detail, list_source):
    #     print('details', detail)
        for tt in list_source:
    #         print(tt.text)
            if detail in tt:
                return tt.replace(detail, '').replace(':', '').strip()
    details = soup.find('div', {'class': 'wc-tab-inner'}).text.strip().split('\n')
    description = soup.find('div', {'class': 'wc-tab-inner'}).text.strip()
    short_description = soup.find('div', {'class': 'woocommerce-product-details__short-description'}).text.strip()
    regex_num = re.compile(r'مكون من (?P<num>\d*\.?\d*) قطع')
    no_of_peices = ''
    p = regex_num.search(short_description)
    if p != None:
        no_of_peices = p['num']
    
    duvit_info = extract_detail('لحاف –', details).replace('سم – قطعة واحدة', '').strip()
    bed_sheet1_info = extract_detail('شرشف', details).replace('سم – قطعة واحدة', '').replace('مطاط – ', '').strip()
    pillow_size = extract_detail('غطاء مخدة', details).replace('سم – أربعة قطع', '').replace('-', '').replace('–', '').strip()
    small_pillow_info = extract_detail('غطاء وسادة', details)
    if small_pillow_info != None:
        small_pillow_info = small_pillow_info.replace('قطعتين', '').replace('سم', '').replace('–', '')
    fabric_type = extract_detail('الغطاء مصنوع من', details)
    stuffing_material = extract_detail('مكونات الحشوة', details)
    barcode = sku
    
    cats = soup.find('nav', {'class': 'woocommerce-breadcrumb'}).find_all('a')
    cat1 = cats[1].text.strip()
    cat2 = cats[2].text.strip()
    data = {
        'sku': sku,
        'name': name,
        'link_url': url,
        'description': description,
        'short_description': short_description,
        'is_in_stock': in_stock,
        'categories1': cat1,
        'categories2': cat2,
        'categories3': '',
        
        'barcode': barcode,
        'no_of_peices': no_of_peices,
        'duvit_info': duvit_info,
        'bed_sheet1_info': bed_sheet1_info,
        'pillow_size': pillow_size,
        'small_pillow_info': small_pillow_info,
        'fabric_type': fabric_type,
        'stuffing_material': stuffing_material,
        
        'price': price,
        
    }
    return data

In [62]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/inrest/categories/inrest_model.xlsx')
for i, url in enumerate(list_product[99:]):
    print('Count:', i)
    
    data = scrape_product(url)
    df1 = pd.DataFrame([data])
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('Inrest_update2.xlsx')
   

Count: 0
Count: 1


In [14]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
cookies = {'session': '134-8225175-0355220'}
url = 'https://inrest.com/product/%d9%85%d9%81%d8%b1%d8%b4-%d9%85%d9%81%d8%b1%d8%af-%d9%85%d9%88%d8%af%d8%b1%d9%86-%d9%84%d9%86%d8%af%d9%86/'
r = requests.get(url, headers=headers, cookies=cookies,  verify=False)

soup  = BeautifulSoup(r.text, 'html.parser')

In [19]:
type_ = soup.find('span', {'class': 'posted_in'}).find_all('a')[-1].text
type_

'مفارش مفرد'

In [ ]:
duvit_info = 'لحاف – 160*230 سم – قطعة واحدة'

In [ ]:
regex = re.compile(r'')

In [22]:
papa = soup.find('div', {'class': 'wc-tab-inner'}).text.split('\n')
papa

['',
 '',
 'مفرش مفرد مودرن لندن',
 'طقم لحاف فاخر مفرد ( لندن ) مكون من 3 قطع, وهي :',
 'لحاف – 160*230 سم – قطعة واحدة',
 'شرشف مطاط – 120*200+25 سم – قطعة واحدة',
 'غطاء مخدة – 48*74+4 سم – قطعة واحدة',
 'الغطاء مصنوع من : 100% مايكروفايبر – بوليستر معالج',
 'مكونات الحشوة : 100% مايكروفايبر – بوليستر معالج',
 '\xa0',
 'لعناية أفضل بمفرش مفرد مودرن لندن',
 'العناية بالمنتج : \xa0',
 '– حرصاً منا للحفاظ على جودة المنتج، يجب غسل المنتج بالبخار (Dry Clean) .',
 '– يمكن غسل بيوت المخدة وشرشف المرتبة بالغسالة العادية .',
 '– ممنوع استخدام المبيضات .',
 '',
 '']

In [31]:
short_description = soup.find('div', {'class': 'woocommerce-product-details__short-description'}).text.strip()
regex_num = re.compile(r'مكون من (?P<num>\d*\.?\d*) قطع')
no_of_peices = ''
p = regex_num.search(short_description)
if p != None:
    no_of_peices = p['num']
print(no_of_peices)

3


In [30]:

print(p['num'])

3


In [20]:
bed_sheet1_info = 'شرشف مطاط – 120*200+25 سم – قطعة واحدة'

In [ ]:
pillow_size = 'غطاء مخدة – 48*74+4 سم – قطعة واحدة'

In [ ]:
small_pillow_info = ''

In [ ]:
fabric_type = 'الغطاء مصنوع من : 100% مايكروفايبر – بوليستر معالج'

In [ ]:
stuffing_material = 'مكونات الحشوة : 100% مايكروفايبر – بوليستر معالج'

In [30]:
price = soup.find('p', {'class': 'price'}).find('span', {'class': 'woocommerce-Price-amount amount'}).text.replace('ر.س','').strip()
price

'989.00'

In [47]:
in_stock = soup.find('p', {'class': 'stock in-stock'}).text.replace('متوفر في المخزون', '').strip()
in_stock

'14'

In [33]:
sku = soup.find('span', {'class': 'sku'}).text.strip()
sku

'AK-HL01-6PCBG'

In [94]:
def extract_detail(detail, list_source):
#     print('details', detail)
    for tt in list_source:
#         print(tt.text)
        if detail in tt:
            return tt.replace(detail, '').replace(':', '').strip()
details = soup.find('div', {'class': 'wc-tab-inner'}).text.strip().split('\n')
description = soup.find('div', {'class': 'wc-tab-inner'}).text.strip()

In [103]:
raw_materials = extract_detail('الغطاء مصنوع من :', details)



In [114]:
details

['غطاء سرير مفرد أوثيلو',
 'غطاء سرير خفيف مضغوط مناسب لأجواء الصيف',
 'طقم غطاء سرير مفرد أوثيلو ( صيفي ) فاخر مكون من 3 قطع',
 'لحاف – 160*210 سم – قطعة واحدة',
 'شرشف مطاط – 120*200+25 سم – قطعة واحدة',
 'غطاء مخدة – 50*75+5 سم – قطعة واحدة',
 'الغطاء مصنوع من : 100% مايكروفايبر – بوليستر معالج',
 'مكونات الحشوة : 100% مايكروفايبر – بوليستر معالج',
 'العناية بالمنتج : \xa0',
 '– امكانية الغسيل بالغسالة المنزلية',
 '– استخدام التجفيف بالهواء الساخن مع درجة الحرارة العادية.',
 '– عندي كي المفرش ، درجة الحرارة المستخدمة كحد اقصى درجة حرارة 30 درجة مئوية.']

In [97]:
raw_materials = extract_detail('الغطاء مصنوع من :', details)
raw_materials

'100% مايكروفايبر – بوليستر معالج'

In [101]:
try:
    garantee = details[retrun_ele_next('الضمان : ', details)].replace('.', '')
    garantee
except:
    garantee = ''

In [102]:
toto = retrun_ele_next('الضمان : ', details)
toto

In [121]:
cats = soup.find('nav', {'class': 'woocommerce-breadcrumb'}).find_all('a')
cat1 =cats[1].text.strip()
cats

[<a class="breadcrumb-link" href="https://inrest.com">الرئيسية</a>,
 <a class="breadcrumb-link" href="https://inrest.com/product-category/31/">المفارش</a>,
 <a class="breadcrumb-link breadcrumb-link-last" href="https://inrest.com/product-category/31/26/">غطاء سرير</a>]

In [123]:
cat1 =cats[1].text.strip()
cat1

'غطاء سرير'